In [2]:
# | default_exp backend.tools.todo

In [3]:
# | export
from agentic.backend.tools.base import Tool
from agentic.backend.schemas import TodoParams, TodoAction
from typing import Dict, Any, List

class TodoTool(Tool):
    def __init__(self):
        self.task_plans = {}  # Store task plans: {task: List[Dict]}

    def get_tool_schema(self) -> Dict[str, Any]:
        """Return the OpenAI-compatible schema for todo."""
        return {
                "type": "function",
                "function": {
                    "name": "todo",
                    "description": "Break down tasks into steps",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "task": {"type": "string"},
                            "action": {"type": "string", "enum": ["plan", "execute", "status"]}
                        },
                        "required": ["task", "action"]
                    }
                }
            }

    def execute(self, **kwargs) -> Dict[str, Any]:
        """Execute todo operations (plan, execute, status)."""
        try:
            params = TodoParams(**kwargs)
            
            if params.action == TodoAction.PLAN:
                steps = self._create_plan(params.task)
                self.task_plans[params.task] = steps
                return {
                    "success": True,
                    "task": params.task,
                    "steps": steps,
                    "message": f"Created plan with {len(steps)} steps"
                }
            
            elif params.action == TodoAction.EXECUTE:
                if params.task not in self.task_plans:
                    return {"error": f"No plan found for task: {params.task}"}
                
                steps = self.task_plans[params.task]
                step_id = params.step_id or 0
                
                if step_id < 0 or step_id >= len(steps):
                    return {"error": f"Invalid step_id: {step_id}"}
                
                step = steps[step_id]
                step["status"] = "completed"
                return {
                    "success": True,
                    "task": params.task,
                    "step": step,
                    "message": f"Executed step {step_id}: {step['description']}"
                }
            
            elif params.action == TodoAction.STATUS:
                if params.task not in self.task_plans:
                    return {"error": f"No plan found for task: {params.task}"}
                
                steps = self.task_plans[params.task]
                completed = sum(1 for step in steps if step.get("status") == "completed")
                return {
                    "success": True,
                    "task": params.task,
                    "steps": steps,
                    "progress": f"{completed}/{len(steps)} steps completed"
                }
            
            return {"error": f"Unknown action: {params.action}"}
            
        except Exception as e:
            return {"error": f"Todo operation failed: {str(e)}"}

    def _create_plan(self, task: str) -> List[Dict]:
        """Create a simple task plan by breaking down the task."""
        # Simplified planning logic (replace with LLM-based planning if needed)
        steps = [
            {"id": 0, "description": f"Start {task}", "status": "pending"},
            {"id": 1, "description": f"Complete {task}", "status": "pending"}
        ]
        return steps